In [ ]:
!pip install rdkit

In [ ]:
!pip install duckdb

In [ ]:
!pip install catboost

## Data Preparation

The training and testing data paths are defined for the .parquet files. We use duckdb to scan search through the large training sets. Just to get started lets sample out an equal number of positive and negatives. 

This query selects an equal number of samples where binds equals 0 (non-binding) and 1 (binding), limited to 30,000 each, to avoid model bias towards a particular class.

In [1]:

# import duckdb
# import pandas as pd

# train_path = '/kaggle/input/leash-predict-chemical-bindings/train.parquet'
# test_path = '/kaggle/input/leash-predict-chemical-bindings/test.parquet'

# con = duckdb.connect()

# df = con.query(f"""(SELECT *
#                         FROM parquet_scan('{train_path}')
#                         WHERE binds = 0
#                         ORDER BY random()
#                         LIMIT 2000000)
#                         UNION ALL
#                         (SELECT *
#                         FROM parquet_scan('{train_path}')
#                         WHERE binds = 1
#                         ORDER BY random()
#                         LIMIT 2000000)""").df()

# con.close()


import modin.pandas as pd
import snowflake.snowpark.modin.plugin
from snowflake.snowpark.session import Session, _get_active_sessions
session = Session.builder.create()

# NOTE the shuffle here isn't guaranteed to actually reorder anything,
# and read_snowflake doesn't necessarily preserve order.
df = pd.read_snowflake(f"""(SELECT *
                        FROM KAGGLE_LEASH_TRAIN
                        WHERE "binds" = 0
                        LIMIT 2000000)
                        UNION ALL
                        (SELECT *
                        FROM KAGGLE_LEASH_TRAIN
                        WHERE "binds" = 1
                        LIMIT 2000000)""")

In [2]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,275847232,O=C(O)C[C@H]1CCCN1C(=O)OCC1c2ccccc2-c2ccccc21,COC1(C(F)(F)CN)CCOCC1.Cl,Cl.NCc1nnc2ncccn12,COC1(C(F)(F)CNc2nc(NCc3nnc4ncccn34)nc(N3CCC[C@...,HSA,1
1,275848138,O=C(O)C[C@H]1CCCN1C(=O)OCC1c2ccccc2-c2ccccc21,COC1(CN)CCC1,CCOC(=O)c1nnc(N)o1,CCOC(=O)c1nnc(Nc2nc(NCC3(OC)CCC3)nc(N3CCC[C@@H...,HSA,1
2,275850238,O=C(O)C[C@H]1CCCN1C(=O)OCC1c2ccccc2-c2ccccc21,COC1(CN)CCOC1.Cl,Cc1cc2cc(CN)ccc2[nH]1,COC1(CNc2nc(NCc3ccc4[nH]c(C)cc4c3)nc(N3CCC[C@@...,HSA,1
3,275850463,O=C(O)C[C@H]1CCCN1C(=O)OCC1c2ccccc2-c2ccccc21,COC1(CN)CCOC1.Cl,Cl.Cl.NCc1cncc(F)c1,COC1(CNc2nc(NCc3cncc(F)c3)nc(N3CCC[C@@H]3CC(=O...,HSA,1
4,275851309,O=C(O)C[C@H]1CCCN1C(=O)OCC1c2ccccc2-c2ccccc21,COC1(CN)CCOC1.Cl,Nc1nc(Cl)c2cn[nH]c2n1,COC1(CNc2nc(Nc3nc(Cl)c4cn[nH]c4n3)nc(N3CCC[C@@...,HSA,1


## Feature Preprocessing

Lets grab the smiles for the fully assembled molecule `molecule_smiles` and generate ecfps for it. We could choose different radiuses or bits, but 2 and 1024 is pretty standard.

In [3]:
from rdkit import Chem
from snowflake.snowpark.functions import udf
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder
import rdkit

pd.session.custom_package_usage_config['enabled'] = True
pd.session.add_packages(["rdkit"])


# # skiping this part since returning Molecule doesn't work.
# @udf
# def udf_mol_from_smiles(molecule_smile: str):
#     from rdkit import Chem
#     return Chem.MolFromSmiles(molecule_smile)

# Convert SMILES to RDKit molecules
# df['molecule'] = df['molecule_smiles'].apply(udf_mol_from_smiles)

# Generate ECFPs
def generate_ecfp(molecule_smile:str, radius=2, bits=1024) -> list:
    if molecule_smile is None:
        return None
    molecule = Chem.MolFromSmiles(molecule_smile)
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

df['ecfp'] = df['molecule_smiles'].apply(generate_ecfp)

convert_dtype is ignored in Snowflake backend.


In [4]:
df

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,ecfp
0,275847232,O=C(O)C[C@H]1CCCN1C(=O)OCC1c2ccccc2-c2ccccc21,COC1(C(F)(F)CN)CCOCC1.Cl,Cl.NCc1nnc2ncccn12,COC1(C(F)(F)CNc2nc(NCc3nnc4ncccn34)nc(N3CCC[C@...,HSA,1,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ..."
1,275848138,O=C(O)C[C@H]1CCCN1C(=O)OCC1c2ccccc2-c2ccccc21,COC1(CN)CCC1,CCOC(=O)c1nnc(N)o1,CCOC(=O)c1nnc(Nc2nc(NCC3(OC)CCC3)nc(N3CCC[C@@H...,HSA,1,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,275850238,O=C(O)C[C@H]1CCCN1C(=O)OCC1c2ccccc2-c2ccccc21,COC1(CN)CCOC1.Cl,Cc1cc2cc(CN)ccc2[nH]1,COC1(CNc2nc(NCc3ccc4[nH]c(C)cc4c3)nc(N3CCC[C@@...,HSA,1,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,275850463,O=C(O)C[C@H]1CCCN1C(=O)OCC1c2ccccc2-c2ccccc21,COC1(CN)CCOC1.Cl,Cl.Cl.NCc1cncc(F)c1,COC1(CNc2nc(NCc3cncc(F)c3)nc(N3CCC[C@@H]3CC(=O...,HSA,1,"[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,275851309,O=C(O)C[C@H]1CCCN1C(=O)OCC1c2ccccc2-c2ccccc21,COC1(CN)CCOC1.Cl,Nc1nc(Cl)c2cn[nH]c2n1,COC1(CNc2nc(Nc3nc(Cl)c4cn[nH]c4n3)nc(N3CCC[C@@...,HSA,1,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...
3589901,83582927,O=C(NC[C@H]1CC[C@H](C(=O)O)CC1)OCC1c2ccccc2-c2...,NC[C@@H]1CCC(=O)N1,C#CCOc1ccc(CN)cc1.Cl,C#CCOc1ccc(CNc2nc(NC[C@@H]3CCC(=O)N3)nc(NC[C@H...,sEH,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3589902,83582945,O=C(NC[C@H]1CC[C@H](C(=O)O)CC1)OCC1c2ccccc2-c2...,NC[C@@H]1CCC(=O)N1,C=CCSCCN,C=CCSCCNc1nc(NC[C@@H]2CCC(=O)N2)nc(NC[C@H]2CC[...,sEH,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3589903,83582951,O=C(NC[C@H]1CC[C@H](C(=O)O)CC1)OCC1c2ccccc2-c2...,NC[C@@H]1CCC(=O)N1,CC(C)(C)NS(=O)(=O)c1cccc(N)c1,CC(C)(C)NS(=O)(=O)c1cccc(Nc2nc(NC[C@@H]3CCC(=O...,sEH,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3589904,83582966,O=C(NC[C@H]1CC[C@H](C(=O)O)CC1)OCC1c2ccccc2-c2...,NC[C@@H]1CCC(=O)N1,CC(C)(C)c1ocnc1CN.Cl,CC(C)(C)c1ocnc1CNc1nc(NC[C@@H]2CCC(=O)N2)nc(NC...,sEH,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."


In [5]:
%time x = df['protein_name'].values

CPU times: user 2.2 s, sys: 247 ms, total: 2.44 s
Wall time: 11.5 s


In [6]:
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(x.reshape(-1, 1))

In [7]:
ecfp_list = df['ecfp'].tolist()

In [8]:
protein_onehot_list = protein_onehot.tolist()

In [10]:
X = [ecfp + protein for ecfp, protein in zip(ecfp_list, protein_onehot_list)]

In [11]:
y = df['binds'].tolist()

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Train Model

In [ ]:
# taking a while so split this into components above to see why.

# # One-hot encode the protein_name
# onehot_encoder = OneHotEncoder(sparse_output=False)
# protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

# # Combine ECFPs and one-hot encoded protein_name
# X = [ecfp + protein for ecfp, protein in zip(df['ecfp'].tolist(), protein_onehot.tolist())]
# y = df['binds'].tolist()

# # Split the data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [13]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,ecfp
0,275847232,O=C(O)C[C@H]1CCCN1C(=O)OCC1c2ccccc2-c2ccccc21,COC1(C(F)(F)CN)CCOCC1.Cl,Cl.NCc1nnc2ncccn12,COC1(C(F)(F)CNc2nc(NCc3nnc4ncccn34)nc(N3CCC[C@...,HSA,1,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ..."
1,275848138,O=C(O)C[C@H]1CCCN1C(=O)OCC1c2ccccc2-c2ccccc21,COC1(CN)CCC1,CCOC(=O)c1nnc(N)o1,CCOC(=O)c1nnc(Nc2nc(NCC3(OC)CCC3)nc(N3CCC[C@@H...,HSA,1,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,275850238,O=C(O)C[C@H]1CCCN1C(=O)OCC1c2ccccc2-c2ccccc21,COC1(CN)CCOC1.Cl,Cc1cc2cc(CN)ccc2[nH]1,COC1(CNc2nc(NCc3ccc4[nH]c(C)cc4c3)nc(N3CCC[C@@...,HSA,1,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,275850463,O=C(O)C[C@H]1CCCN1C(=O)OCC1c2ccccc2-c2ccccc21,COC1(CN)CCOC1.Cl,Cl.Cl.NCc1cncc(F)c1,COC1(CNc2nc(NCc3cncc(F)c3)nc(N3CCC[C@@H]3CC(=O...,HSA,1,"[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,275851309,O=C(O)C[C@H]1CCCN1C(=O)OCC1c2ccccc2-c2ccccc21,COC1(CN)CCOC1.Cl,Nc1nc(Cl)c2cn[nH]c2n1,COC1(CNc2nc(Nc3nc(Cl)c4cn[nH]c4n3)nc(N3CCC[C@@...,HSA,1,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


# Best Score received with CatBoostClassifier

In [14]:
from catboost import CatBoostClassifier
catb_model = CatBoostClassifier().fit(X_train, y_train)
y_pred_proba = catb_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")

Learning rate set to 0.291616
0:	learn: 0.5986995	total: 247ms	remaining: 4m 6s
1:	learn: 0.5401163	total: 449ms	remaining: 3m 44s
2:	learn: 0.5085983	total: 615ms	remaining: 3m 24s
3:	learn: 0.4878249	total: 794ms	remaining: 3m 17s
4:	learn: 0.4704625	total: 982ms	remaining: 3m 15s
5:	learn: 0.4555153	total: 1.16s	remaining: 3m 11s
6:	learn: 0.4411462	total: 1.34s	remaining: 3m 9s
7:	learn: 0.4263969	total: 1.5s	remaining: 3m 6s
8:	learn: 0.4161750	total: 1.66s	remaining: 3m 2s
9:	learn: 0.4063730	total: 1.82s	remaining: 3m
10:	learn: 0.3980559	total: 2s	remaining: 2m 59s
11:	learn: 0.3888516	total: 2.19s	remaining: 3m
12:	learn: 0.3808901	total: 2.36s	remaining: 2m 59s
13:	learn: 0.3718499	total: 2.54s	remaining: 2m 58s
14:	learn: 0.3634304	total: 2.71s	remaining: 2m 58s
15:	learn: 0.3553946	total: 2.9s	remaining: 2m 58s
16:	learn: 0.3492386	total: 3.08s	remaining: 2m 57s
17:	learn: 0.3444157	total: 3.23s	remaining: 2m 56s
18:	learn: 0.3379185	total: 3.4s	remaining: 2m 55s
19:	learn:

Look at that Average Precision score. We did amazing! 

Actually no, we just overfit. This is likely recurring theme for this competition. It is easy to predict molecules that come from the same corner of chemical space, but generalizing to new molecules is extremely difficult.

## Test Prediction

 The trained Random Forest model is then used to predict the binding probabilities. These predictions are saved to a CSV file, which serves as the submission file for the Kaggle competition.

In [ ]:
# import os

# # Process the test.parquet file chunk by chunk
# test_file = '/kaggle/input/leash-predict-chemical-bindings/test.csv'
# output_file = 'submission.csv'  # Specify the path and filename for the output file

# # Read the test.parquet file into a pandas DataFrame
# for df_test in pd.read_csv(test_file, chunksize=100000):

#     # Generate ECFPs for the molecule_smiles
#     df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
#     df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)

#     # One-hot encode the protein_name
#     protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))

#     # Combine ECFPs and one-hot encoded protein_name
#     X_test = [ecfp + protein for ecfp, protein in zip(df_test['ecfp'].tolist(), protein_onehot.tolist())]

#     # Predict the probabilities
#     probabilities = catb_model.predict_proba(X_test)[:, 1]

#     # Create a DataFrame with 'id' and 'probability' columns
#     output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})

#     # Save the output DataFrame to a CSV file
#     output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))


In [16]:
import os

# Process the test.parquet file chunk by chunk
df_test = pd.read_snowflake('KAGGLE_LEASH_TEST')
output_file = 'submission.csv'  # Specify the path and filename for the output file



In [18]:
df_test['ecfp'] = df_test['molecule_smiles'].apply(generate_ecfp)

In [19]:
protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))

In [20]:
X_test = [ecfp + protein for ecfp, protein in zip(df_test['ecfp'].tolist(), protein_onehot.tolist())]

In [21]:
probabilities = catb_model.predict_proba(X_test)[:, 1]

In [22]:
output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})

In [24]:
output_df.to_pandas().to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))